### Case study of XAI portfolio building and optimization

This notebook gives an overview of the steps taken for portfolio optimization using AI and leveraging XAI. The portfolio optimization notebook works with openly available datasets, but any other data can be used as long as its compatible. The steps for the portfolio building and optimizations are as follows:

        1. Install and import necessary libraries
        2. Declare constans and vairables
        3. Setup dataset
        4. Setup goals (Portfolio size and expected performance)
        5. Construct portfolio 
        6. Masure performance
        7. Using explainability

#### Dependencies

In [ ]:
import os
from os import path
import yfinance as yf
import pandas as pd
from datetime import datetime
import shap
import lime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.optimize import minimize
from sklearn.ensemble import RandomForestRegressor
import shap
import tensorflow as tf

#### Constants and utility functions

In [ ]:
START_DATE  = datetime.strptime("2010-01-01", "%Y-%m-%d").date()
SPLIT_DATE  = datetime.strptime("2019-01-01", "%Y-%m-%d").date()
END_DATE    = datetime.strptime("2022-01-01", "%Y-%m-%d").date()
LABEL       = "Close"
WINDOW      = 50

def create_dataset(dataset, window):
    dataX, dataY = [], []
    for i in range(len(dataset)-window-1):
        a = dataset[i:(i+window), :]
        dataX.append(a)
        dataY.append(dataset[i + window, :])
    return np.array(dataX), np.array(dataY)


def create_portfolios(df,num_portfolios,mean_daily_returns,cov_matrix):
    num_symbols = len(df['Symbol'].unique())
    results = np.zeros((3 + num_symbols, num_portfolios))
    for i in range(num_portfolios):
        weights = np.array(np.random.random(num_symbols))
        weights /= np.sum(weights)
        portfolio_return = np.sum(mean_daily_returns * weights) * 252
        portfolio_std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
        results[0, i] = portfolio_return
        results[1, i] = portfolio_std_dev
        # Store Sharpe Ratio (return / volatility) - risk free rate element excluded for simplicity
        results[2, i] = results[0, i] / results[1, i]
        for j in range(len(weights)):
            results[j+3, i] = weights[j]
    results_frame = pd.DataFrame(results.T, columns=['return', 'stdev', 'sharpe'] + [str(i) for i in range(num_symbols)])
    return results_frame

#### Download stocks and assets

In this notebook we use publicly available datasets. One from www.kaggle.com for SP500 stocks (https://www.kaggle.com/datasets/andrewmvd/sp-500-stocks) and additionally yahoo finance (https://pypi.org/project/yfinance/) for other assets.
The dataset SP500 needs to be downloaded from the source first.
The dataset with commodities is downloaded from yfinance.

In [ ]:
#Loading data


#select 10 stocks from sp500 to reduce dataset for faster execution of this notebook

selected = ['AAPL','MSFT','AMZN','NVDA','GOOGL','TSLA','JPM','WMT','PG','PFE']
dateparse = lambda dates: datetime.strptime(dates, '%Y-%m-%d')
iter_csv = pd.read_csv(path.join(os.getcwd(),'sp500_stocks.csv'), iterator=True, chunksize=1000, parse_dates = ['Date'], index_col = 'Date', date_parser = dateparse)
df = pd.concat([chunk[chunk['Symbol'].isin(selected)] for chunk in iter_csv])



# Downloading gold, oil, natural gas, currencies, Bitcoin
other = ['GC=F','CL=F','NG=F','GBPUSD=X','EURUSD=X','BTC-USD']
all = []
for c in other:
    cmd = yf.download(c, start=START_DATE, end=END_DATE, interval='1d')
    cmd['Symbol'] = c
    all.append(cmd)

In [ ]:
#Preprocess data
dfOther = pd.concat(all)
df = pd.concat([df, dfOther])
df = df.sort_values('Date')
df.fillna(method='bfill', inplace=True)
df = df.reset_index()
df.head(5)


In [ ]:
# Pivot table per symbol
pivot_df = df.pivot(columns='Symbol', values=LABEL)
# Daily returns
returns = pivot_df.pct_change()
# Mean returns and the covariance matrix of returns
mean_daily_returns = returns.mean()
cov_matrix = returns.cov()



#### Portfolio selection using random forest and explaining using SHAP

We chose a number of portfolios to be randomly selected and allocated from the data
After this we calculate the returns and volatility of each portfolio
Then we use ML approach (random forest) to predict the sharpe value of each portfolio
In this way we can select the best portfolio by using sharpe value as metric.
Finally we select the 10 best performing portfolios and use LIME to explain each of them.


In [ ]:
num_portfolios = 10000

results_frame = create_portfolios(pivot_df,num_portfolios,mean_daily_returns,cov_matrix)

# Define dataset for ML
X = results_frame.drop('sharpe', axis=1)
y = results_frame['sharpe']

# Fit model using RandomForestRegressor
model = RandomForestRegressor(n_estimators=100)
model.fit(X, y)

#Explain using SHAP
shap.initjs()
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)
shap.summary_plot(shap_values, X, plot_type="bar")

In [ ]:
# Select top portfolios:

# Locate position of portfolio with highest Sharpe Ratio
max_sharpe_port = results_frame.iloc[results_frame['sharpe'].idxmax()]
# Locate positon of portfolio with minimum standard deviation
min_vol_port = results_frame.iloc[results_frame['stdev'].idxmin()]

# Compute SHAP values for top 10 portfolios
shap_values = explainer.shap_values([max_sharpe_port,min_vol_port])

# Plot the SHAP values
shap.summary_plot(shap_values, [max_sharpe_port,min_vol_port], plot_type="bar")

In [ ]:
# Create a LimeTabularExplainer
explainer = lime.lime_tabular.LimeTabularExplainer(X.values, feature_names=X.columns, class_names=['sharpe'], verbose=True, mode='regression')

# Explain each of the top 10 portfolios
for i in range(len(top_portfolios)):
    exp = explainer.explain_instance(top_portfolios.iloc[i], model.predict, num_features=5)
    print('Portfolio', i)
    exp.show_in_notebook(show_table=True)

## Portfolio selection using LSTM and XAI

We use LSTM to predict returns and use XAI to explain the predictions

In [ ]:
#Preprocess again
# Pivot table per symbol
pivot_df = df.pivot(columns='Symbol', values=LABEL)

# Daily returns
returns = pivot_df.pct_change()
returns = returns.dropna()
# Scale data to fit for NN approach
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(returns)


In [ ]:
#Split data to train and test set 
split_ratio = 0.9
train_data = scaled_data[:int(split_ratio*len(scaled_data))]
test_data = scaled_data[int(split_ratio*len(scaled_data)):]

In [ ]:

# Create training and test datasets
trainX, trainY = create_dataset(train_data, WINDOW)
testX, testY = create_dataset(test_data, WINDOW)

In [ ]:
# Create and fit the LSTM network
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(trainX.shape[1], trainX.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=trainY.shape[1]))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(trainX, trainY, epochs=10, batch_size=16)

In [ ]:
#Evaluate model

trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

plt.figure(figsize=(16,8))
plt.plot(testY[0], color='blue', label='Actual return')
plt.plot(testPredict[:,0], color='red', label='Predicted return')
plt.title('Return Prediction')
plt.xlabel('Time')
plt.ylabel('Return')
plt.legend()
plt.show()


In [ ]:
#Explain model

# Create a LimeTabularExplainer
explainer = lime_tabular.LimeTabularExplainer(trainX.reshape(trainX.shape[0], -1), 
                                              feature_names = [f"Day {i+1}" for i in range(trainX.shape[1])], 
                                              class_names=['Return'], 
                                              verbose=True, 
                                              mode='regression')

# Explain a prediction (explain single instance of prediction)
exp = explainer.explain_instance(testX[0].reshape(-1), model.predict, num_features=5)
exp.show_in_notebook(show_table=True)

### Using LSTM predicted returns to optimize portfolio

This time we use SHAP to explain, but we use kernel explainer which is adequate for LSTM

In [ ]:
mean_daily_returns = testPredict.mean(axis=0)
cov_matrix = np.cov(testPredict.T)
num_portfolios = 10000

results_frame = create_portfolios(pivot_df,num_portfolios,mean_daily_returns,cov_matrix)

#Visualize 
plt.scatter(results_frame.stdev, results_frame.return, c=results_frame.sharpe, cmap='RdYlBu')
plt.colorbar()

In [ ]:
# Locate position of portfolio with highest Sharpe Ratio
max_sharpe_port = results_frame.iloc[results_frame['sharpe'].idxmax()]
# Locate positon of portfolio with minimum standard deviation
min_vol_port = results_frame.iloc[results_frame['stdev'].idxmin()]

#Explain using SHAP
explainer = shap.KernelExplainer(model.predict, trainX)
# Compute SHAP values for a single prediction (e.g., first prediction)
shap_values = explainer.shap_values(testX[0])
# Plot the SHAP values
shap.force_plot(explainer.expected_value[0], shap_values[0], testX[0])